In [24]:
import mido, rtmidi, rtmidi_, os

In [4]:
import config

In [5]:
mid = mido.MidiFile(config.dataset_dir + 'mary.mid')

In [21]:
messages = []
# for msg in mid.play(): # play with rests
for msg in mid:
    messages.append(msg)
    
for msg in messages[:4]:
    print(msg)

<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
control_change channel=0 control=91 value=58 time=0
control_change channel=0 control=10 value=69 time=0


In [23]:
msg

<message control_change channel=0 control=10 value=69 time=0>

In [53]:
dirname = config.dataset_dir + 'lmd_matched/A/A/A/'
folder = dirname + os.listdir(dirname)[-1]
filename = folder + '/' + os.listdir(folder)[-1]
mid = mido.MidiFile(filename)
filename

'../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid'

In [54]:
mid

<midi file '../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid' type 1, 14 tracks, 11010 messages>

In [55]:
filename

'../datasets/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/dfea6fd75926c571a87db789280d059d.mid'

In [60]:
for x in os.walk(config.dataset_dir + 'lmd_matched/'):
    x = x

In [68]:
fn, _, filelist = x
fn + ' / - ' + filelist[0]

'../datasets/lmd_matched/Z/Z/Z/TRZZZYV128F92E996D / - 1b966417a9aa703873c5fa1cfe18da32.mid'

In [73]:
dirname = config.dataset_dir + 'examples/'
filenames = os.listdir(dirname)[:2]
midis = []
for fn in filenames:
    mid = mido.MidiFile(dirname + fn)
    midis.append(mid)

In [79]:
for msg in midis[0]:
    print(msg)

<meta message track_name name='01 16th Snare\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=36 velocity=104 time=0
note_on channel=0 note=38 velocity=57 time=0
note_off channel=0 note=38 velocity=64 time=0.08333333333333333
note_off channel=0 note=36 velocity=64 time=0.020833333333333332
note_on channel=0 note=38 velocity=73 time=0.010416666666666666
note_off channel=0 note=38 velocity=64 time=0.08333333333333333
note_on channel=0 note=38 velocity=107 time=0.041666666666666664
note_off channel=0 note=38 velocity=64 time=0.109375
note_on channel=0 note=38 velocity=104 time=0.020833333333333332
note_off channel=0 note=38 velocity=64 time=0.09895833333333333
note_on channel=0 note=38 velocity=64 time=0.020833333333333332
note_on channel=0 note=36 velocity=103 time=0.00520833333

In [80]:
msg

<meta message end_of_track time=0>